In [16]:
$ServersCSVFilePath = "$pwd\servers.csv"
$AppsCSVFilePath    = "$pwd\applications.csv"
$expCSV  = "$pwd\users_apps.csv"

In [26]:
function generateListOfServersFromCSVFile($filePath)
{
    try{
        Write-Host "[+] Reading file: $filePath"
        $csvServers = Import-Csv $filePath -UseCulture
    } 

    catch {
        Write-Host "A Fatal error occurred opening $filePath"
        Write-Host $_
        exit
    }
    
    $ServerList = New-Object Collections.Generic.List[string]
    foreach ($i in $csvServers) { 
        $ServerList.Add($i.'Server Name')
    }
    return $ServerList

}

function generateListofAppsFromCSVFile($filePath){
    try{
        Write-Host "[+] Reading file: $filePath"
        $csvApps = Import-Csv $filePath -UseCulture
    }

    catch {
        Write-Host "A Fata; error occurred opening $filePath"
        Write-Host $_
        exit
    }

    return $csvApps
}
    
$ServerList = generateListOfServersFromCSVFile($ServersCSVFilePath)
$csvApps = generateListofAppsFromCSVFile($AppsCSVFilePath)

$csvContents = Invoke-Command -ComputerName $ServerList -ScriptBlock {
###############
# This whole section is passed as a script-block to invoke command

    function printCSV($a){
        $line = $a.Server + "`t" + $a.Date + "`t" + $a.Time + "`t" + $a.Application + "`t" + $a.User + "`t" + $a.Memory + "`t" + $a.Path + "`r`n"
        return $line
        #$a.values | export-csv
    }

    function convertAndFormatInMB($size){
        $size = $size
        $size = ($size/1MB)
        $size = $size.tostring("##.##")
        return $size
    }

    function getProcessInfoFor32B($process) {

        $memInMB = convertAndFormatInMB($process.PrivateMemorySize[0] -as[int])
        $procInfo   = @{
        Server      = $env:computername
        Date        = Get-Date -UFormat "%m/%d/%Y" 
        Time        = $process.StartTime
        Application = $process.Name
        User        = $process.UserName
        Memory      = $memInMB + "MB"
        Path        = $process.Path
        }

        return $procInfo
    }

    function getProcessInfoFor64B($process) {
        $memInMB = convertAndFormatInMB($process.PrivateMemorySize[0] -as[int])
        $procInfo   = @{
        Server      = $env:computername
        Date        = Get-Date -UFormat "%m/%d/%Y" 
        Time        = $process.StartTime[0]
        Application = $process.Name[0]
        User        = $process.UserName[0]
        Memory      = $memInMB + "MB"
        Path        = $process.Path[0]
        }
        return $ProcInfo
    }


    function findMatchingProcessesAndReturnCSVFormattedRows($applicationList)
    {
        # Get one process list to search through and then re-use it for performance
        $procList = Get-Process -IncludeUserName
        
        foreach ($myProc in $list) {
            foreach ($i in $applicationList) { 
                $applicationName = $i.Application
                if ( $procList.Name -contains $applicationName) {
                    # 32b programs return a string for their attributes instead of an object
                    if ($myProc.Path.GetType().Name -eq 'String') {

                        $procInfo = getProcessInfoFor32B($myProc)

                    } else {

                       $procInfo = getProcessInfoFor64B($myProc)
                    }

                #$rows += printCSV($procInfo)
                printCSV($procInfo)
                $procInfo = @{}
                }
            }
        }
        #return $rows
    }

    # The $using:csvAPPS passes a local variable to the scriptblock as $csvAPPS

    findMatchingProcessesAndReturnCSVFormattedRows($using:csvAPPS)

    ## End Scriptblock
    }


[+] Reading file: C:\Users\ansible\Downloads\users_apps\servers.csv
[+] Reading file: C:\Users\ansible\Downloads\users_apps\applications.csv


In [27]:
$csvHeader="Server`tDate`tTime`tApplication`tUser`tMemory`tPath`r`n"
#$csvHeader
if ($csvContents) {
    $csv = $csvHeader
    $csv += $csvContents
    if ($Append) {
        $csvContents | out-file $expCSV -Append  -Encoding UTF8
        Write-Host "[+] Successfully appended to" $expCSV
    }
    else {
        $csv | out-file $expCSV  -Encoding UTF8
        Write-Host "[+] Successfully exported to" $expCSV
        write-host $cvs
    }
}

. {
>> $csvHeader="Server`tDate`tTime`tApplication`tUser`tMemory`tPath`r`n"
>> #$csvHeader
>> if ($csvContents) {
>>     $csv = $csvHeader
>>     $csv += $csvContents
>>     if ($Append) {
>>         $csvContents | out-file $expCSV -Append  -Encoding UTF8
>>         Write-Host "[+] Successfully appended to" $expCSV
>>     }
>>     else {
>>         $csv | out-file $expCSV  -Encoding UTF8
>>         Write-Host "[+] Successfully exported to" $expCSV
>>         write-host $cvs
>>     }
>> }
>> }
>> 


In [25]:
     function printCSV($a){
        $line = $a.Server + "`t" + $a.Date + "`t" + $a.Time + "`t" + $a.Application + "`t" + $a.User + "`t" + $a.Memory + "`t" + $a.Path + "`r`n"
        return $line
        #$a.values | export-csv
    }

    function convertAndFormatInMB($size){
        $size = $size
        $size = ($size/1MB)
        $size = $size.tostring("##.##")
        return $size
    }

    function getProcessInfoFor32B($process) {

        $memInMB = convertAndFormatInMB($process.PrivateMemorySize[0] -as[int])
        $procInfo   = @{
        Server      = $env:computername
        Date        = Get-Date -UFormat "%m/%d/%Y" 
        Time        = $process.StartTime
        Application = $process.Name
        User        = $process.UserName
        Memory      = $memInMB + "MB"
        Path        = $process.Path
        }

        return $procInfo
    }

    function getProcessInfoFor64B($process) {
        $memInMB = convertAndFormatInMB($process.PrivateMemorySize[0] -as[int])
        $procInfo   = @{
        Server      = $env:computername
        Date        = Get-Date -UFormat "%m/%d/%Y" 
        Time        = $process.StartTime[0]
        Application = $process.Name[0]
        User        = $process.UserName[0]
        Memory      = $memInMB + "MB"
        Path        = $process.Path[0]
        }
        return $ProcInfo
    }
  
  function findMatchingProcessesAndReturnCSVFormattedRows($applicationList)
    {
        # Get one process list to search through and then re-use it for performance
        $procList = Get-Process -IncludeUserName
        
        foreach ($myProc in $list) {
            foreach ($i in $applicationList) { 
                $applicationName = $i.Application
                if ( $procList.Name -contains $applicationName) {
                    # 32b programs return a string for their attributes instead of an object
                    if ($myProc.Path.GetType().Name -eq 'String') {

                        $procInfo = getProcessInfoFor32B($myProc)

                    } else {

                       $procInfo = getProcessInfoFor64B($myProc)
                    }

                #$rows += printCSV($procInfo)
                printCSV($procInfo)
                $procInfo = @{}
                }
            }
        }
        #return $rows
    }

findMatchingProcessesAndReturnCSVFormattedRows($csvAPPS)


. {
>>      function printCSV($a){
>>         $line = $a.Server + "`t" + $a.Date + "`t" + $a.Time + "`t" + $a.Application + "`t" + $a.User + "`t" + $a.Memory + "`t" + $a.Path + "`r`n"
>>         return $line
>>         #$a.values | export-csv
>>     }
>> 
>>     function convertAndFormatInMB($size){
>>         $size = $size
>>         $size = ($size/1MB)
>>         $size = $size.tostring("##.##")
>>         return $size
>>     }
>> 
>>     function getProcessInfoFor32B($process) {
>> 
>>         $memInMB = convertAndFormatInMB($process.PrivateMemorySize[0] -as[int])
>>         $procInfo   = @{
>>         Server      = $env:computername
>>         Date        = Get-Date -UFormat "%m/%d/%Y" 
>>         Time        = $process.StartTime
>>         Application = $process.Name
>>         User        = $process.UserName
>>         Memory      = $memInMB + "MB"
>>         Path        = $process.Path
>>         }
>> 
>>         return $procInfo
>>     }
>> 
>>     function getProcessInfoFor64B(

In [11]:
Get-Process | gm



   TypeName: System.Diagnostics.Process

Name                       MemberType     Definition                                                                   
----                       ----------     ----------                                                                   
Handles                    AliasProperty  Handles = Handlecount                                                        
Name                       AliasProperty  Name = ProcessName                                                           
NPM                        AliasProperty  NPM = NonpagedSystemMemorySize64                                             
PM                         AliasProperty  PM = PagedMemorySize64                                                       
SI                         AliasProperty  SI = SessionId                                                               
VM                         AliasProperty  VM = VirtualMemorySize64                                                   

Company                    ScriptProperty System.Object Company {get=$this.Mainmodule.FileVersionInfo.CompanyName;}    
CPU                        ScriptProperty System.Object CPU {get=$this.TotalProcessorTime.TotalSeconds;}               
Description                ScriptProperty System.Object Description {get=$this.Mainmodule.FileVersionInfo.FileDescri...
FileVersion                ScriptProperty System.Object FileVersion {get=$this.Mainmodule.FileVersionInfo.FileVersion;}
Path                       ScriptProperty System.Object Path {get=$this.Mainmodule.FileName;}                          
Product                    ScriptProperty System.Object Product {get=$this.Mainmodule.FileVersionInfo.ProductName;}    
ProductVersion             ScriptProperty System.Object ProductVersion {get=$this.Mainmodule.FileVersionInfo.Product...


